In [3]:
# Creating a user interface so that the user can enter 
# their own initial state for the puzzle.
import math
MAX_SIZE = 100

def main():
    # TODO: Right now it fails if user make mistakes
    # Check size to be an integer square - 1, use try except on rows input
    square_less_1 = [x**2-1 for x in range(2, 11, 1)]
    size = int(input("This is an N-puzzle solver. Please choose size of game " \
                     f"(can only be N^2-1, e.g. {square_less_1}) no larger than {MAX_SIZE}:"))
    if size > MAX_SIZE:
        print("Size is too large, please try again")
        return
    if size not in square_less_1:
        print('Size game entered is not an integer square - 1, please try again')
        return
    N = int(math.sqrt(size+1))
    puzzle_type = input("Press 1 to create your own puzzle or 2 for a default puzzle:")
    if puzzle_type == '1':
        print("When entering your puzzle, make sure to enter 0 for the blank space and use spaces between numbers:")
        # Creating an empty NxN matrix for the user to create the initial state
        try:
            mat = [N*[0] for i in range(N)]
            # Iterating over each row in the matrix
            for i in range(N):
                mat[i] = input(f"Enter row {str(i)}: ")
            #print(mat)
            # Since the input the user gives for the puzzle is a string,
            # we need to convert the elements of this puzzle to integers.
            for j in range(N):
                splt_row = mat[j].split()
                mat[j] = [int(splt_row[k]) for k in range(N)]
#                 print(mat[j])
        except:
            print("Please pay attention to the format required")
            return
    if puzzle_type == '2':
        pass
        # call function that will run search with default puzzle. 
        # maybe give options for the difficulty of default puzzle.
        
    print(f"This is the starting state:")
    for i in range(N):
        print(mat[i])
        
    # Ask user for:
    # Algorithm
    # Would you like a print of the best sequence of moves y/n - into PRINT_MOVES
    # Would you like a full trace of the algorithm? NOTE: this could be pretty long - into PRINT_TRACE

In [4]:
main()

This is an N-puzzle solver. Please choose size of game (can only be N^2-1, e.g. [3, 8, 15, 24, 35, 48, 63, 80, 99]) no larger than 100:


ValueError: invalid literal for int() with base 10: ''

In [6]:
# Creating a function that will create the goal state based
# on the number of dimensions that the user gives it.
# Arguments:
# N: The number of rows and columns for the square matrix.
def puzzle(N):
    # Creating an matrix of dimension NxN with all 0's
    mat = [N*[0] for i in range(N)]
    # Iterating over all rows
    for r in range(N):
        # For each row, iterating over each column
        for c in range(N):
            #print(r,c)
            # Assigning the corresponding value of the puzzle
            # by using the following formula.
            mat[r][c] = N*r + c%N + 1
            
    # Based on the formula used for each element above, the [N,N] position
    # will contain (N+1) but we want it to be 0 to represent an empty space
    mat[N-1][N-1] = 0
    
    return mat

In [6]:
puzzle(2)

[[1, 2], [3, 0]]

In [7]:
a = puzzle(3)
a[1]

[4, 5, 6]

In [8]:
import copy
b = copy.deepcopy(a)
b[0][0]=100
b

[[100, 2, 3], [4, 5, 6], [7, 8, 0]]

In [9]:
def mm(mat_src):
    mat = copy.deepcopy(mat_src)
    mat[0][0] = 200
    return mat

In [10]:
b = mm(a)

In [11]:
a

[[1, 2, 3], [4, 5, 6], [7, 8, 0]]

In [12]:
b

[[200, 2, 3], [4, 5, 6], [7, 8, 0]]

In [7]:
# This function creates a dictionary with the correct indices for each of 
# the tiles in the given state.
def mat2dict(mat):
    # Defining an intial empty dictiory
    goal_dict = {}
    # The dimension of mat
    n = len(mat)
    # Iterating over each element in the matrix
    for i in range(n):
        for j in range(n):
            # The value of the i,j tile
            tile = mat[i][j]
            #print(tile)
            # For each tile, assign its corresponding indices
            goal_dict[tile] = [i,j]
    
    return goal_dict

In [33]:
def mat2tpl(mat):
    """Convert 2-D array to tuple of tuples"""
    return tuple([tuple(x) for x in mat])

In [14]:
# Testing the function mat2dict
mat2dict(puzzle(2))

{0: [1, 1], 1: [0, 0], 2: [0, 1], 3: [1, 0]}

In [119]:
import heapq as heap

# Creating a function for the A* algorithm that will work for 
# uniform cost search, the misplaced tile heuristic and the 
# manhatten distance heuristic.
# The arguments for the function are:
# goal_mat: The goal state for the puzzle
# start_mat: The puzzle that we are starting the search with.
# g: A cost function
# h: A heuristic function
def astar(goal_mat, start_mat, h, g = 1):
    print("I have entered A*")
    # At the beginning of the search, first check whether or not
    # the start_mat is the goal state 
    if goal_state(start_mat, goal_mat) == True:
        return "I have reached the goal state!" # not sure what to return yet
    
    # Initializing the list that will contain the priorirty queue of all the states
    # where each state will have a tuple (cost, count, current state, moves)
    pq = []
    # Initializing a count variable that will be used for comparison
    # in pq in the case that the costs of two states are equivalent.
    count = 0
    # Pushing start_mat to the heap. Assigning a cost of 0 to the initial state
    heap.heappush(pq, (0, count, start_mat, []))
    
    # Initializing the dictionary that will keep track of each nodes cost and update
    # as the node is revisited. This will make sure that we don't revisit a node with higher cost.
    add_dict = {}
    
    # Keep searching until pq is an empty list
    while len(pq) != 0:
        # Popping off the node in the priority queue and assigning 
        # variables to each of the components in the tuple which will be used throughout
        crnt_cost, _, crnt_state, crnt_mvs = heap.heappop(pq)
        #print(crnt_mvs)
        # At each iteration check to see if the prev_state is the goal state or not 
        #print(mat2dict(crnt_state)[0])
        if goal_state(crnt_state, goal_mat) == True:
            # Creating a dictionary that will keep track of the cost, the moves from the initial state to 
            # goal state, the depth (length of moves list), and the number of nodes expanded (length of add_dict)
            otpt = {"cost": crnt_cost, "moves": crnt_mvs, "depth": len(crnt_mvs), "nodes_exp": len(add_dict)}
            return otpt # TODO: might need to change this later
        
        # In order to not repeat states with higher cost, adding a condition here
        # to check whether or not the crnt_cost is greater than the cost in add_dict
        # which always contains the minimum cost. If this is true don't expand the current 
        # node and go back to the beginning of the while loop.
        if mat2tpl(crnt_state) in add_dict and crnt_cost > add_dict[mat2tpl(crnt_state)]:
            print("I have reached an added node with higher cost")
            continue
        
#         print(crnt_state)
        # TODO: Need to add some kind of stopping condition for count just in case
        # the search grows too large.
        if count > 10000000:
            print(count)
            return "I have been searching for too long"
        
        # Find the possible moves from the current state
        moves = next_move(crnt_state)
#         print(moves)
        #print(moves)
        # For each move in moves, find all the child nodes of the current state.
        for mv in moves:
            child = get_next_state(crnt_state, mv)
            # cost of the child node which includes the cost of the current state
            child_cost = g + h(child, goal_mat) + crnt_cost
            # TODO: Need to include some sort of code here to account for the visited
            # nodes
            
            # At each move, if the child node has already been visited, 
            # we want to check if the cost of the child node is smaller than previously 
            # and update it in the dictionary
            if mat2tpl(child) not in add_dict or child_cost <= add_dict[mat2tpl(child)]:
                add_dict[mat2tpl(child)] = child_cost
                # count increases by 1 for each child node
                count += 1
            
                #print(mv)
                # Pushing the child node into the heap making sure to include
                # the crnt_mvs to mv so that later I can recreate the path to the goal state.
                heap.heappush(pq, (child_cost, count, child, crnt_mvs+[mv]))
                                                                            
    # print(count)
    return       

In [120]:
# Testing the astar function where the initial state is the goal state
#print(astar(goal_mat = puzzle(3), start_mat = puzzle(3), h = h_0))

# Testing astar where the initial state is simple
st_1 = [[1,2,3],[4,5,6],[7,0,8]]
#print(astar(goal_mat = puzzle(3), start_mat = init_state, h = h_0))

init_state = [[0, 7, 2],
 [4, 6, 1],
 [3, 5, 8]]
#print(astar(goal_mat = puzzle(3), start_mat = init_state, h = h_manhatt))
astar(goal_mat = puzzle(3), start_mat = init_state, h = h_manhatt)

I have entered A*


{'cost': 234,
 'depth': 28,
 'moves': [((0, 1), (0, 0)),
  ((0, 2), (0, 1)),
  ((1, 2), (0, 2)),
  ((1, 1), (1, 2)),
  ((2, 1), (1, 1)),
  ((2, 0), (2, 1)),
  ((1, 0), (2, 0)),
  ((1, 1), (1, 0)),
  ((2, 1), (1, 1)),
  ((2, 2), (2, 1)),
  ((1, 2), (2, 2)),
  ((1, 1), (1, 2)),
  ((0, 1), (1, 1)),
  ((0, 2), (0, 1)),
  ((1, 2), (0, 2)),
  ((2, 2), (1, 2)),
  ((2, 1), (2, 2)),
  ((2, 0), (2, 1)),
  ((1, 0), (2, 0)),
  ((0, 0), (1, 0)),
  ((0, 1), (0, 0)),
  ((1, 1), (0, 1)),
  ((2, 1), (1, 1)),
  ((2, 0), (2, 1)),
  ((1, 0), (2, 0)),
  ((1, 1), (1, 0)),
  ((2, 1), (1, 1)),
  ((2, 2), (2, 1))],
 'nodes_exp': 18559}

In [32]:
d = {}
crnt = [[1, 2], [3, 4]]
crnt_tpl = tuple([tuple(e) for e in crnt])
#d[l] = 1
crnt_tpl

((1, 2), (3, 4))

In [170]:
get_next_state(crnt = init_state, move = ((2,2),(2,1)))

[[1, 2, 3], [4, 5, 6], [7, 8, 0]]

In [71]:
li = {2: 'ddd', 3: 'lafjdsakf'}
if 3 in li: print('dd')

dd


In [133]:
s = (None, 0, 's', [])
crnt = s
moves = ['m1']
for move in moves:
    new = (3, 1, 'A', crnt[3]+[move])
new

(3, 1, 'A', ['m1'])

In [134]:
crnt = new
moves = ['m2']
for move in moves:
    new = (3, 1, 'B', crnt[3]+[move])
new

(3, 1, 'B', ['m1', 'm2'])

In [135]:
['m1']+['m2']

['m1', 'm2']

In [115]:
import heapq as heap
def heapsort(iterable):
    h = []
    for value in iterable:
        heap.heappush(h, value)
        print(h)
    return [heap.heappop(h) for i in range(len(h))]

heapsort([1, 3, 5, 7, 9, 2, 4, 6, 8, 0])


[1]
[1, 3]
[1, 3, 5]
[1, 3, 5, 7]
[1, 3, 5, 7, 9]
[1, 3, 2, 7, 9, 5]
[1, 3, 2, 7, 9, 5, 4]
[1, 3, 2, 6, 9, 5, 4, 7]
[1, 3, 2, 6, 9, 5, 4, 7, 8]
[0, 1, 2, 6, 3, 5, 4, 7, 8, 9]


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
l = [(3,1), (2,2), (5,2)]
heap.heappush(l, (344,1))
for num in l: heap.heappush(l, num)
    
#heap.heappush(l, (2,2))
#heap.heappush(l, (5,2))
#heap.heappop(l)

In [9]:
def h_0(crnt, goal):
    return 0

In [80]:
def h_misplace(crnt, goal):
    # number of rows and columns in crnt
    n = len(crnt)
    # Counter for the number of misplaced tiles
    misplace = 0
    # iterating over all elements in the crnt and goal matrices
    for i in range(n):
        for j in range(n):
            #print(crnt[i][j], goal[i][j])
            # Checking to see if the i,j element in crnt state 
            # is in the right place or not. Including a condition
            # that the misplaced element can't equal 0 since this
            # heuristic doesn't consider it as misplaced.
            if crnt[i][j] != goal[i][j] and crnt[i][j] != 0:
                # Each time we have a misplaced tile, misplace increases by 1
                misplace += 1
    return misplace

In [11]:
state = [[1,2,4],[3,0,6],[7,8,5]]
h_misplace(state, puzzle(3))

1 1
2 2
4 3
3 4
0 5
6 6
7 7
8 8
5 0


3

In [118]:
def h_manhatt(crnt, goal):
    # Dimension of crnt and goal
    n = len(crnt)
    # Creating a variable for the manhattan distance
    manhatt_dist = 0
    # Iterating over all elements in the matrix
    for i in range(n):
        for j in range(n):
            # Don't want to calculate manhattan distance for tile 0
            if crnt[i][j] != 0:
                # The value of the i,j tile in crnt
                tile = crnt[i][j]
                # The correct indices based on the goal state
                # for the value tile
                crct_ind = mat2dict(goal)[tile]
                # Calculating the manhattan distance
                manhatt_dist += abs(i - crct_ind[0]) + abs(j - crct_ind[1])
                #print(manhatt_dist)
    return manhatt_dist

In [115]:
# Testing h_manhatt
state = [[1,2,3],[4,5,6],[7,8,0]]
state
h_manhatt(state, puzzle(3))
#state

0
0
0
0
0
0
0
0


0

In [114]:
state

[[1, 2, 3], [5, 4, 6], [7, 8, 0]]

In [14]:
# This function determines the next possible moves based on the 
# current state in the search.
def next_move(crnt):
    # The dimension of crnt
    N = len(crnt)
    # Creating a dictionary of pieces and their corresponding indices
    # so that we can identify the indices of 0, the blank space.
    crnt_dict = mat2dict(crnt)
    # The indices of the blank space 0
    blank_ind = tuple(crnt_dict[0])
    # Creting an empty list to keep track of the possible moves 
    # from the blank space
    moves_from = []
    
    #print(blank_ind)
    
    # If the row index doesn't equal 0 (the first row), we can make a move from the top
    if blank_ind[0] != 0: moves_from.append('T')
    # If the row index doesn't equal N-1 (the last row), we can make a move from the bottom
    if blank_ind[0] != N-1: moves_from.append('B')
    # If the column index doesn't equal 0 (the first column), we can make a move from the left
    if blank_ind[1] != 0: moves_from.append('L')
    # If the column index doesn't equal N-1 (the last column), we can make a move from the right
    if blank_ind[1] != N-1: moves_from.append('R')
    
    #print(moves_from)
    
    # Creating an empty list to keep track of the indices that the blank space 
    # can move to
    moves = []
        
    # Determining the indices based on the possible moves in the list moves_from
    for d in moves_from:
        # If we can make a move from the top, then the row index of the blank space
        # decreases by 1 and the column index stays the same.
        if d == 'T': moves.append(((blank_ind[0]-1,blank_ind[1]), blank_ind))
        # If we can make a move from the bottom, then the row index of the blank space
        # increases by 1 and the column index stays the same. 
        elif d == 'B': moves.append(((blank_ind[0]+1,blank_ind[1]), blank_ind))
        # If we can make a move from the left, then the column index of the blank space
        # decreases by 1 and the row index stays the same. 
        elif d == 'L': moves.append(((blank_ind[0],blank_ind[1]-1), blank_ind))
        # If we can make a move from the right, then the column index of the blank space
        # increases by 1 and the row index stays the same. 
        elif d == 'R': moves.append(((blank_ind[0],blank_ind[1]+1), blank_ind))
    
    return moves

In [15]:
# Testing the next_move function
state = [[1,7,0],[5,2,6],[3,8,4]]
next_move(state)

[((1, 2), (0, 2)), ((0, 1), (0, 2))]

In [19]:
import copy
"""
This function creates the next states (child nodes) from the given
current state. 
Arguments - 
crnt: A matrix of the current state in the search.
move: A move from the current state.
"""
def get_next_state(crnt, move):
    # Creating a deep copy of the current state so that I can
    # update the child node properly without modifying crnt.
    child = copy.deepcopy(crnt)
    # Switching the values of the blank space in the current state 
    # with the value that we are moving.
    child[move[0][0]][move[0][1]] = crnt[move[1][0]][move[1][1]] # the new blank space
    child[move[1][0]][move[1][1]] = crnt[move[0][0]][move[0][1]] # the moved piece
    
    return child

In [61]:
# Testing get_next_state function
state = [[8,0,1],[6,7,2],[5,4,3]]
mv = next_move(state)
for move in mv:
    print(move)
    print(get_next_state(state,move))

((1, 1), (0, 1))
[[8, 7, 1], [6, 0, 2], [5, 4, 3]]
((0, 0), (0, 1))
[[0, 8, 1], [6, 7, 2], [5, 4, 3]]
((0, 2), (0, 1))
[[8, 1, 0], [6, 7, 2], [5, 4, 3]]


In [18]:
# Function that determines whether or not the search has 
# reached the goal state given the current state.
# Arguments:
# crnt: The current state in the search.
# goal: the goal state of the search.
# Return:
# 
def goal_state(crnt, goal):
    # Compares the crnt and goal matrices to determine if they 
    # are equivalent.
    return(crnt == goal)